# Step 1: Define the Base Network
# The base network generates embeddings for input samples. For simplicity, we use a convolutional neural network (CNN).

In [5]:
from tensorflow.keras import layers, models

def create_base_network(input_shape):
    """
    Creates a base network for feature extraction.
    """
    input = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(64, activation='relu')(x)
    return models.Model(input, x)

# Step 2: Construct the Siamese Network
# The Siamese network compares two inputs by passing them through the base network (shared weights).

In [3]:
from tensorflow.keras import backend as K

def siamese_network(input_shape):
    """
    Constructs the Siamese network.
    """
    # Define the inputs
    input_a = layers.Input(shape=input_shape)
    input_b = layers.Input(shape=input_shape)

    # Create the shared base network
    base_network = create_base_network(input_shape)

    # Generate embeddings for both inputs
    embedding_a = base_network(input_a)
    embedding_b = base_network(input_b)

    # Compute the L2 distance between the embeddings
    l2_distance = layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([embedding_a, embedding_b])

    # Add a dense layer to output the similarity score
    output = layers.Dense(1, activation='sigmoid')(l2_distance)

    # Create the Siamese model
    model = models.Model(inputs=[input_a, input_b], outputs=output)
    return model

# Step 3: Contrastive Loss Function
# Contrastive loss helps the model learn to minimize the distance for similar pairs and maximize it for dissimilar pairs.

In [4]:
import tensorflow as tf

def contrastive_loss(y_true, y_pred):
    """
    Contrastive loss function.
    """
    margin = 1
    return tf.reduce_mean(y_true * tf.square(y_pred) + (1 - y_true) * tf.square(tf.maximum(margin - y_pred, 0)))


# Step 4: Training the Model
# Compile and train the Siamese network.

In [ ]:
# Define the input shape
input_shape = (105, 105, 1)  # Example for grayscale images

# Instantiate the Siamese network
siamese_model = siamese_network(input_shape)

# Compile the model
siamese_model.compile(optimizer='adam', loss=contrastive_loss, metrics=['accuracy'])

# Prepare training data (pairs of inputs and labels)
# X_train_a, X_train_b = pairs of images
# y_train = similarity labels (1 or 0)

# Train the model
siamese_model.fit(
    [X_train_a, X_train_b],
    y_train,
    batch_size=32,
    epochs=10,
    validation_data=([X_val_a, X_val_b], y_val)
)

# Step 5: Evaluate and Use the Model

In [ ]:
# Evaluate the model
loss, accuracy = siamese_model.evaluate([X_test_a, X_test_b], y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# Make predictions
predictions = siamese_model.predict([X_sample_a, X_sample_b])
print(f"Similarity Score: {predictions}")